In [199]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import math
from sklearn.model_selection import train_test_split
from scipy import stats
from keras import backend as K
from keras.callbacks import TensorBoard
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout
from keras.layers import TimeDistributed
from keras.layers import Activation
from keras.layers import MaxPooling1D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import Conv2D
from keras.optimizers import SGD
from keras.utils import to_categorical
from keras.utils import plot_model
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
import numpy as np
import os
import matplotlib.pyplot as plt
import random as rand
import pandas as pd
import tensorflow
import keras
import statistics as st
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation
from keras.optimizers import SGD
from keras.utils.vis_utils import plot_model
from sklearn.preprocessing import LabelEncoder
from collections import Counter


In [200]:
os.chdir("/Users/carterloftus/Documents/ECS 289/")

In [ ]:
### Functions for F1 score (obtained online, have to cite), which we will use in the models later
def f1(y_true, y_pred):
    def recall(y_true, y_pred):
        """Recall metric.

        Only computes a batch-wise average of recall.

        Computes the recall, a metric for multi-label classification of
        how many relevant items are selected.
        """
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
        recall = true_positives / (possible_positives + K.epsilon())
        return recall

    def precision(y_true, y_pred):
        """Precision metric.

        Only computes a batch-wise average of precision.

        Computes the precision, a metric for multi-label classification of
        how many selected items are relevant.
        """
        true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
        predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
        precision = true_positives / (predicted_positives + K.epsilon())
        return precision
    precision = precision(y_true, y_pred)
    recall = recall(y_true, y_pred)
    return 2*((precision*recall)/(precision+recall+K.epsilon()))

In [365]:
training_data = pd.read_csv('train_df.csv')
val_data = pd.read_csv('val_df.csv')

### Remove unncessary columns
data_train = training_data.iloc[:,[4,5,6,7,8,9,10,11,13]]
data_val = val_data.iloc[:,[4,5,6,7,8,9,10,11,13]]

### Change -1 labels to 0
#data_train.loc[(data_train.loc[:,'LABEL_O']== -1),'LABEL_O'] = 0
#data_val.loc[(data_val.loc[:,'LABEL_O']== -1),'LABEL_O'] = 0

### Or just drop the unknown class (-1)
data_train = data_train[~data_train['LABEL_O'].isin([-1])]
data_val = data_val[~data_val['LABEL_O'].isin([-1])]
### And then reduce the value of the other categories to start at 0
data_train['LABEL_O'] = data_train['LABEL_O']-1
data_val['LABEL_O'] = data_val['LABEL_O']-1

### Standardize the features
def standardize(data,col_nums_to_standardize):
    for col in col_nums_to_standardize:
        data.iloc[:,col] = (data.iloc[:,col] - np.mean(data.iloc[:,col]))/np.std(data.iloc[:,col])
    return data

data_train = standardize(data_train,col_nums_to_standardize = range(7))
data_val = standardize(data_val,col_nums_to_standardize = range(7))

## Extremely basis NN with no aggregation of time:

epochs, batch_size, verbose = 10,64,1


train_x = np.array(data_train.iloc[:,:-2])
train_y = np.array(data_train.iloc[:,-2])
train_y = to_categorical(train_y)
val_x = np.array(data_val.iloc[:,:-2])
val_y = np.array(data_val.iloc[:,-2])
val_y = to_categorical(val_y)

def build_basicDNN():
    model = Sequential()
    model.add(Dense(100,activation = 'relu',input_dim = train_x.shape[1]))
    model.add(Dropout(0.5))
    model.add(Dense(100,activation='relu'))
    model.add(Dense(num_classes, activation='softmax'))
    return model
    
def train_model(model):
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    model.fit(train_x,train_y,epochs=epochs,batch_size=batch_size,verbose=verbose)
    loss,accuracy = model.evaluate(val_x, val_y, batch_size=batch_size, verbose=verbose)
    #final_loss,final_acc = model.evaluate(test_features_x, test_features_y, batch_size = batch_size, verbose=verbose)
    return loss,accuracy # . ,final_loss,final_acc

model = build_basicDNN()
loss,accuracy = train_model(model)

Epoch 1/10
120576/120576 [==============================] - 12s 98us/step - loss: 0.6151 - acc: 0.7834
Epoch 2/10
120576/120576 [==============================] - 8s 66us/step - loss: 0.5862 - acc: 0.7894
Epoch 3/10
120576/120576 [==============================] - 8s 66us/step - loss: 0.5740 - acc: 0.7914
Epoch 4/10
120576/120576 [==============================] - 8s 64us/step - loss: 0.5647 - acc: 0.7932
Epoch 5/10
120576/120576 [==============================] - 8s 70us/step - loss: 0.5568 - acc: 0.7951
Epoch 6/10
120576/120576 [==============================] - 9s 74us/step - loss: 0.5503 - acc: 0.7971
Epoch 7/10
120576/120576 [==============================] - 10s 84us/step - loss: 0.5430 - acc: 0.7979
Epoch 8/10
120576/120576 [==============================] - 10s 79us/step - loss: 0.5378 - acc: 0.7996
Epoch 9/10
120576/120576 [==============================] - 9s 76us/step - loss: 0.5339 - acc: 0.8011
Epoch 10/10
30108/30108 [==============================] - 2s 75us/step


In [320]:
#Break features into groups of size n
group_size = 6

def break_into_samples(features,group_size):
    l = []
    sep_ts = set(features.loc[:,'ID'])
    for ts in sep_ts:
        sub_data  = np.array(features.loc[features.loc[:,'ID']== ts,:])
        a = int(sub_data.shape[0]/group_size)
        if a != 0:
            sub_data = sub_data[:(a*group_size),:]
            sub_data = [np.split(sub_data,a)]
            l = l + sub_data
    features = np.vstack(l)
    features = features[:,:,:-1]
    return features

data_train = break_into_samples(data_train,group_size)
data_val = break_into_samples(data_val,group_size)

In [347]:
### If already working with a separated training/validation set, skip down to the next function (split_features_label()), but if not, uncomment this and run it
## def split_dataset_train_test(features,thresh = int(features.shape[0]*0.75), many_to_one = True, prop = 0.65):
#    np.random.shuffle(features)
#    train_x = features[:thresh,:,:features.shape[2]-1]
#    train_y = features[:thresh,:,features.shape[2]-1]
#    if many_to_one == True:
#         y_temp = np.zeros(train_y.shape[0])
#         y_temp.fill(np.nan)
#         for i in range(y.shape[0]):
#             row = train_y[i,:]
#             trans_class = (Counter(row)).most_common(1)[0]
#             if trans_class[1] < group_size*prop : 
#                 y_temp[i] = 4
#             else:
#                 y_temp[i] = st.mode(row)
#         train_y = y_temp
#    num_classes = len(set(train_y))
#    train_y = to_categorical(train_y,num_classes)
#    val_x = features[thresh:,:,:features.shape[2]-1]
#    val_y = features[thresh:,:,features.shape[2]-1]
#    if many_to_one == True:
#         y_temp = np.zeros(val_y.shape[0])
#         y_temp.fill(np.nan)
#         for i in range(val_y.shape[0]):
#             row = val_y[i,:]
#             trans_class = (Counter(row)).most_common(1)[0]
#             if trans_class[1] < group_size*prop : 
#                 y_temp[i] = 4
#             else:
#                 y_temp[i] = st.mode(row)
#         val_y = y_temp
#    val_y = to_categorical(train_y,num_classes)
#    return train_x, train_y, val_x, val_y,num_classes


def split_features_label(features,many_to_one = True,prop = 0.65):
    np.random.shuffle(features)
    x = features[:,:,:features.shape[2]-1]
    y = features[:,:,features.shape[2]-1]
    if many_to_one == True:
        y_temp = np.zeros(y.shape[0])
        y_temp.fill(np.nan)
        for i in range(y.shape[0]):
            row = y[i,:]
            trans_class = (Counter(row)).most_common(1)[0]
            if trans_class[1] < group_size*prop : 
                y_temp[i] = 4
            else:
                y_temp[i] = st.mode(row)
        y = y_temp
    
    num_classes = len(set(y))
    y = to_categorical(y,num_classes)
    return x,y,num_classes

train_x,train_y,num_classes = split_features_label(data_train)
val_x,val_y,num_classes = split_features_label(data_val)

### Build a basic feedforward DNN by aggregating all inputs in each time segment:

train_x_flat = train_x.reshape(train_x.shape[0],(train_x.shape[1] * train_x.shape[2]))
val_x_flat = val_x.reshape(val_x.shape[0],(val_x.shape[1] * val_x.shape[2]))

def build_DNN():
    model = Sequential()
    model.add(Dense(100,activation = 'relu',input_dim = train_x_flat.shape[1]))
    model.add(Dropout(0.5))
    model.add(Dense(100,activation='relu'))
    model.add(Dense(num_classes, activation='softmax'))
    return model
    
def train_model(model):
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    model.fit(train_x_flat,train_y,epochs=epochs,batch_size=batch_size,verbose=verbose)
    loss,accuracy = model.evaluate(val_x_flat, val_y, batch_size=batch_size, verbose=verbose)
    #final_loss,final_acc = model.evaluate(test_features_x, test_features_y, batch_size = batch_size, verbose=verbose)
    return loss,accuracy # . ,final_loss,final_acc

model = build_DNN()
loss,accuracy = train_model(model)

Epoch 1/10
20096/20096 [==============================] - 2s 99us/step - loss: 1.9924 - acc: 0.6881
Epoch 2/10
20096/20096 [==============================] - 1s 26us/step - loss: 0.8564 - acc: 0.7290
Epoch 3/10
20096/20096 [==============================] - 1s 25us/step - loss: 0.8152 - acc: 0.7293
Epoch 4/10
20096/20096 [==============================] - 0s 25us/step - loss: 0.8010 - acc: 0.7295
Epoch 5/10
20096/20096 [==============================] - 0s 22us/step - loss: 0.7944 - acc: 0.7295
Epoch 6/10
20096/20096 [==============================] - 0s 22us/step - loss: 0.7903 - acc: 0.7295
Epoch 7/10
20096/20096 [==============================] - 1s 31us/step - loss: 0.7850 - acc: 0.7295
Epoch 8/10
20096/20096 [==============================] - 0s 24us/step - loss: 0.7813 - acc: 0.7296
Epoch 9/10
20096/20096 [==============================] - 1s 26us/step - loss: 0.7749 - acc: 0.7296
Epoch 10/10
5018/5018 [==============================] - 1s 121us/step


In [336]:
### Now train an LSTM

n_timesteps, n_features, n_outputs = train_x.shape[1], train_x.shape[2], train_y.shape[1]
n_sub_groups = 1
n_sub_group_timesteps = int(n_timesteps/n_sub_groups)

def split_samples_into_sub_samples(features):
    features = features.reshape((features.shape[0],n_sub_groups,n_sub_group_timesteps,n_features))
    return features


train_x = split_samples_into_sub_samples(train_x)
val_x = split_samples_into_sub_samples(val_x)



def build_model():
    model = Sequential()
    #Define CNN Model
    model.add(TimeDistributed(Conv1D(filters=64, kernel_size=2, activation="relu"), input_shape=(None,n_sub_group_timesteps,n_features)))
    model.add(TimeDistributed(Conv1D(filters=64, kernel_size=2, activation = "relu")))
    model.add(TimeDistributed(Dropout(0.5)))
    model.add(TimeDistributed(MaxPooling1D(pool_size=2)))
    model.add(TimeDistributed(Flatten()))

    #Define LSTM Model
    model.add(LSTM(100))
    #model.add(LSTM(100, input_shape=(n_timesteps,n_features)))
    model.add(Dropout(0.5))
    model.add(Dense(100, activation='relu'))
    model.add(Dense(n_outputs, activation='softmax'))
    return model

def train_model(model):
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    model.fit(train_x,train_y,epochs=epochs,batch_size=batch_size,verbose=verbose)
    loss,accuracy = model.evaluate(val_x, val_y, batch_size=batch_size, verbose=verbose)
    #final_loss,final_acc = model.evaluate(test_features_x, test_features_y, batch_size = batch_size, verbose=verbose)
    return loss,accuracy # . ,final_loss,final_acc

model = build_model()
loss,accuracy = train_model(model)

print("Validation loss: ",[loss,accuracy])
# print("Final Testing loss: ",[final_loss,final_acc])

Epoch 1/10
20096/20096 [==============================] - 4s 210us/step - loss: 0.8268 - acc: 0.7237
Epoch 2/10
20096/20096 [==============================] - 2s 75us/step - loss: 0.6910 - acc: 0.7647
Epoch 3/10
20096/20096 [==============================] - 1s 73us/step - loss: 0.6450 - acc: 0.7796
Epoch 4/10
20096/20096 [==============================] - 2s 77us/step - loss: 0.6318 - acc: 0.7826
Epoch 5/10
20096/20096 [==============================] - 2s 80us/step - loss: 0.6281 - acc: 0.7844
Epoch 6/10
20096/20096 [==============================] - 2s 76us/step - loss: 0.6220 - acc: 0.7861
Epoch 7/10
20096/20096 [==============================] - 2s 76us/step - loss: 0.6215 - acc: 0.7859
Epoch 8/10
20096/20096 [==============================] - 2s 82us/step - loss: 0.6167 - acc: 0.7850
Epoch 9/10
20096/20096 [==============================] - 2s 80us/step - loss: 0.6157 - acc: 0.7868
Epoch 10/10
5018/5018 [==============================] - 1s 142us/step
Validation loss:  [0.7661825

In [368]:
### Two methods of plotting (first one seems better)

### For plots:
history = model.fit(train_x, train_y,
                epochs=50,
                batch_size=128,
                shuffle=True,
                validation_data=(val_x, val_y),
                callbacks=[TensorBoard(log_dir='/tmp/model2')])

### Find plot at TensorBoard

# Plot training & validation accuracy values
plt.plot(history.history['acc'])
plt.plot(history.history['val_acc'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')
plt.show()

# Plot training & validation loss values
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')
plt.show()

print("Validation loss: ",[loss,accuracy])
# print("Final Testing loss: ",[final_loss,final_acc])

plot_model(model, to_file='model.png')



Train on 120576 samples, validate on 30108 samples
Epoch 1/50
120576/120576 [==============================] - 6s 49us/step - loss: 0.5217 - acc: 0.8040 - val_loss: 0.7410 - val_acc: 0.7446
Epoch 2/50
120576/120576 [==============================] - 6s 50us/step - loss: 0.5193 - acc: 0.8053 - val_loss: 0.7285 - val_acc: 0.7523
Epoch 3/50
120576/120576 [==============================] - 6s 47us/step - loss: 0.5174 - acc: 0.8048 - val_loss: 0.7155 - val_acc: 0.7585
Epoch 4/50
110208/120576 [==========================>...] - ETA: 0s - loss: 0.5159 - acc: 0.8059

KeyboardInterrupt: 